# Adaptive Gate 3-Way Comparison

**Purpose**: Compare three field conditions to determine if learned field gating improves performance:

- **field_on**: Normal field (baseline) - agents always read full field
- **field_off**: Instant decay = no field information
- **adaptive**: Learned gate modulates field usage per channel

**Key question**: Does letting agents *learn* when to use the field beat always-on or always-off?

**Metrics tracked**:
- Reward comparison (mean, 95% CI, statistical tests)
- Gate evolution over training (do gates tend toward 0 or 1?)
- Gate bimodality (do agents specialize into field-users vs field-ignorers?)

## Setup
1. Runtime > Change runtime type > **TPU v6e** + **High-RAM**
2. Run all cells (Ctrl+F9)

In [ ]:
# Cell 1: Setup - Mount Drive, clone repo, install
from google.colab import drive
drive.mount('/content/drive')

import os
REPO_DIR = '/content/emergence-lab'
GITHUB_USERNAME = "imashishkh21"  # Verified from git remote

if not os.path.exists(REPO_DIR):
    !git clone https://github.com/{GITHUB_USERNAME}/emergence-lab.git {REPO_DIR}
else:
    !cd {REPO_DIR} && git pull

os.chdir(REPO_DIR)
!pip install -e ".[dev]" -q
!pip install scipy -q  # For statistical tests
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Cell 2: Imports + TPU/GPU Verification
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import gc
import math
import traceback
from scipy import stats
from collections import defaultdict

from src.configs import Config, TrainingMode
from src.training.parallel_train import ParallelTrainer
from src.agents.network import ActorCritic
from src.agents.policy import sample_actions
from src.environment.env import reset, step
from src.environment.obs import get_observations

# TPU/GPU verification
print(f"JAX version: {jax.__version__}")
print(f"Devices: {jax.devices()}")
device_str = str(jax.devices()[0])
assert 'TPU' in device_str or 'GPU' in device_str, f"No accelerator! Found: {device_str}"
print("Accelerator check passed.")

In [ ]:
# Cell 3: Constants & Paths
DRIVE_BASE = '/content/drive/MyDrive/emergence-lab/adaptive_gate_test'
os.makedirs(DRIVE_BASE, exist_ok=True)

TOTAL_STEPS = 2_000_000  # Quick test (full = 10M)
NUM_ENVS = 32
NUM_STEPS = 128
MAX_AGENTS = 64
SEEDS = [42, 123, 456]
CONDITIONS = ['field_on', 'field_off', 'adaptive']
NUM_EVAL_EPISODES = 5

STEPS_PER_ITER = NUM_ENVS * NUM_STEPS * MAX_AGENTS
NUM_ITERATIONS = math.ceil(TOTAL_STEPS / STEPS_PER_ITER)

print(f"Drive base: {DRIVE_BASE}")
print(f"Total steps: {TOTAL_STEPS:,}")
print(f"Steps per iteration: {STEPS_PER_ITER:,}")
print(f"Num iterations: {NUM_ITERATIONS}")
print(f"Seeds: {SEEDS}")
print(f"Conditions: {CONDITIONS}")

In [ ]:
# Cell 4: Config Builders
# Base config with v2 sweep values - shared by all conditions

def build_base_config() -> Config:
    """Build base config with v2 sweep values."""
    cfg = Config()
    # Environment
    cfg.env.grid_size = 40
    cfg.env.num_agents = 16
    cfg.env.num_food = 25
    cfg.env.max_steps = 500
    # Evolution (survival-friendly)
    cfg.evolution.enabled = True
    cfg.evolution.food_energy = 100
    cfg.evolution.starting_energy = 200
    cfg.evolution.max_energy = 300
    cfg.evolution.reproduce_threshold = 180
    cfg.evolution.reproduce_cost = 80
    cfg.evolution.energy_per_step = 1
    cfg.evolution.max_agents = MAX_AGENTS
    # Field (v2 sweep values)
    cfg.field.num_channels = 4
    cfg.field.channel_diffusion_rates = (0.7, 0.01, 0.0, 0.0)
    cfg.field.channel_decay_rates = (0.02, 0.0001, 0.0, 0.0)
    cfg.field.territory_write_strength = 0.02
    cfg.field.field_value_cap = 1.0
    cfg.field.adaptive_gate = False  # Default: no gating
    # Nest
    cfg.nest.radius = 4
    cfg.nest.compass_noise_rate = 0.2
    # Training
    cfg.train.training_mode = TrainingMode.GRADIENT
    cfg.train.num_envs = NUM_ENVS
    cfg.train.num_steps = NUM_STEPS
    cfg.train.total_steps = TOTAL_STEPS
    cfg.log.wandb = False
    cfg.log.save_interval = 0
    return cfg


def build_field_off_config() -> Config:
    """Build Field OFF config: instant decay = no field information."""
    cfg = build_base_config()
    cfg.field.channel_diffusion_rates = (0.0, 0.0, 0.0, 0.0)
    cfg.field.channel_decay_rates = (1.0, 1.0, 1.0, 1.0)
    cfg.field.territory_write_strength = 0.0
    return cfg


def build_adaptive_config() -> Config:
    """Build adaptive gate config: agents learn when to use field."""
    cfg = build_base_config()
    cfg.field.adaptive_gate = True
    return cfg


# Print config summary
print("="*70)
print("CONFIG SUMMARY")
print("="*70)
for name, builder in [("field_on", build_base_config), 
                       ("field_off", build_field_off_config), 
                       ("adaptive", build_adaptive_config)]:
    cfg = builder()
    print(f"\n{name}:")
    print(f"  adaptive_gate: {cfg.field.adaptive_gate}")
    print(f"  channel_decay_rates: {cfg.field.channel_decay_rates}")
    print(f"  territory_write_strength: {cfg.field.territory_write_strength}")

In [ ]:
# Cell 5: Evaluation Function

def run_eval(network, params, config, key, use_gate_bias=False, num_steps=500):
    """Run a single eval episode using lax.scan.

    Args:
        network: ActorCritic network.
        params: Network parameters.
        config: Environment config.
        key: PRNG key.
        use_gate_bias: If True and config.field.adaptive_gate is True,
                       pass agent_gate_bias to sample_actions.
        num_steps: Number of eval steps.

    Returns:
        Dict with total_reward, final_population, trail_strength, and
        gate_bias_values (if adaptive and use_gate_bias=True).
    """
    key, reset_key = jax.random.split(key)
    init_state = reset(reset_key, config)

    def _eval_step(carry, _unused):
        state, rng, total_reward = carry
        obs = get_observations(state, config)
        obs_batched = obs[None, :, :]  # (1, max_agents, obs_dim)
        rng, act_key = jax.random.split(rng)

        # Pass gate_bias for adaptive config
        gb = None
        if use_gate_bias and config.field.adaptive_gate and state.agent_gate_bias is not None:
            gb = state.agent_gate_bias[None, :, :]  # (1, max_agents, num_channels)

        actions, _, _, _, _ = sample_actions(network, params, obs_batched, act_key, gb)
        actions = actions[0]
        state, rewards, done, info = step(state, actions, config)
        alive = state.agent_alive.astype(jnp.float32)
        total_reward = total_reward + jnp.sum(rewards * alive)
        return (state, rng, total_reward), None

    (final_state, _, total_reward), _ = jax.lax.scan(
        _eval_step, (init_state, key, jnp.float32(0.0)), None, length=num_steps,
    )

    # Trail strength from Ch0
    ch0 = jnp.asarray(final_state.field_state.values[:, :, 0])
    nonzero_mask = ch0 > 0.01
    trail_strength = jnp.where(
        jnp.any(nonzero_mask),
        jnp.sum(jnp.where(nonzero_mask, ch0, 0.0)) / jnp.maximum(jnp.sum(nonzero_mask.astype(jnp.float32)), 1.0),
        0.0,
    )

    result = {
        'total_reward': float(total_reward),
        'final_population': int(jnp.sum(final_state.agent_alive)),
        'trail_strength': float(trail_strength),
    }

    # For adaptive: capture final EVOLVED gate biases (genotype)
    # Note: Actual gate output = sigmoid(learned_weights * hidden + bias)
    # This captures the evolved preference, not momentary gate activation
    if config.field.adaptive_gate and final_state.agent_gate_bias is not None:
        alive_mask = final_state.agent_alive
        gate_vals = final_state.agent_gate_bias[alive_mask]  # (n_alive, num_channels)
        result['gate_bias_values'] = np.array(gate_vals)  # Renamed for clarity

    return result

print("Evaluation function ready.")

In [ ]:
# Cell 6: Training Loop
# Main training loop with resume safety

RESULTS_PATH = os.path.join(DRIVE_BASE, 'all_results.pkl')
if os.path.exists(RESULTS_PATH):
    with open(RESULTS_PATH, 'rb') as f:
        all_results = pickle.load(f)
else:
    all_results = []

completed = {(r['condition'], r['seed_id']) for r in all_results if r.get('success')}

CONFIG_BUILDERS = {
    'field_on': build_base_config,
    'field_off': build_field_off_config,
    'adaptive': build_adaptive_config,
}

for condition in CONDITIONS:
    for seed_id in SEEDS:
        if (condition, seed_id) in completed:
            print(f"[{condition}|seed={seed_id}] SKIPPED (already completed)")
            continue

        config = CONFIG_BUILDERS[condition]()
        checkpoint_dir = os.path.join(DRIVE_BASE, f'{condition}_seed{seed_id}')

        print(f"\n{'='*60}")
        print(f"Training: {condition} | Seed: {seed_id}")
        print(f"{'='*60}")

        try:
            trainer = ParallelTrainer(
                config=config, num_seeds=1, seed_ids=[seed_id],
                checkpoint_dir=checkpoint_dir, master_seed=seed_id,
            )
            train_metrics = trainer.train(
                num_iterations=NUM_ITERATIONS,
                checkpoint_interval_minutes=30,
                resume=True, print_interval=5,
            )

            # Create network for eval
            network = ActorCritic(
                hidden_dims=tuple(config.agent.hidden_dims),
                num_actions=config.agent.num_actions,
                adaptive_gate=config.field.adaptive_gate,
                num_field_channels=config.field.num_channels,
            )
            # Extract params for this seed (parallel trainer uses stacked params)
            seed_params = jax.tree.map(lambda x: x[0], trainer._parallel_state.params)

            # Run eval episodes
            eval_results = []
            all_gate_bias_values = []
            for ep in range(NUM_EVAL_EPISODES):
                key = jax.random.PRNGKey(seed_id * 1000 + ep)
                res = run_eval(network, seed_params, config, key, use_gate_bias=True)
                eval_results.append(res)
                if 'gate_bias_values' in res:
                    all_gate_bias_values.append(res['gate_bias_values'])

            result = {
                'condition': condition,
                'seed_id': seed_id,
                'success': True,
                'total_reward': float(np.mean([r['total_reward'] for r in eval_results])),
                'final_population': float(np.mean([r['final_population'] for r in eval_results])),
                'trail_strength': float(np.mean([r['trail_strength'] for r in eval_results])),
                'train_metrics': train_metrics,  # For learning curves
            }

            if all_gate_bias_values:
                result['gate_bias_values'] = np.concatenate(all_gate_bias_values, axis=0)

            all_results.append(result)
            print(f"  Eval: reward={result['total_reward']:.1f}, pop={result['final_population']:.1f}")

        except Exception as e:
            traceback.print_exc()
            all_results.append({
                'condition': condition, 'seed_id': seed_id,
                'error': str(e), 'success': False,
            })
        finally:
            try:
                del trainer
            except NameError:
                pass
            gc.collect()
            jax.clear_caches()

        # Atomic save
        tmp = RESULTS_PATH + '.tmp'
        with open(tmp, 'wb') as f:
            pickle.dump(all_results, f, protocol=pickle.HIGHEST_PROTOCOL)
        os.replace(tmp, RESULTS_PATH)

print(f"\nCompleted: {len([r for r in all_results if r.get('success')])} / {len(CONDITIONS) * len(SEEDS)}")

In [ ]:
# Cell 7: Group Results by Condition + Summary

# Note: ParallelTrainer.train() only returns FINAL metrics, not the full
# training history. Learning curves would require modifying ParallelTrainer
# to return all_metrics instead of final_metrics, or logging to a separate file.

colors = {'field_on': '#2ecc71', 'field_off': '#e74c3c', 'adaptive': '#3498db'}

by_condition = defaultdict(list)
for r in all_results:
    if r.get('success'):
        by_condition[r['condition']].append(r)

print("=" * 60)
print("RESULTS BY CONDITION")
print("=" * 60)
for condition, runs in by_condition.items():
    print(f"\n{condition} ({len(runs)} successful runs):")
    for run in runs:
        print(f"  Seed {run['seed_id']}: reward={run['total_reward']:.1f}, pop={run['final_population']:.1f}")

# Learning curves note
print()
print("Note: Learning curves not available in this version.")
print("ParallelTrainer returns only final metrics, not training history.")
print("To enable curves, modify ParallelTrainer.train() to return all_metrics.")

In [ ]:
# Cell 8: Aggregate Results & Summary Table

summary = {}
for cond, runs in by_condition.items():
    rewards = [r['total_reward'] for r in runs]
    pops = [r['final_population'] for r in runs]
    n = len(rewards)
    summary[cond] = {
        'reward_mean': np.mean(rewards),
        'reward_std': np.std(rewards, ddof=1) if n > 1 else 0,
        'reward_ci': 1.96 * np.std(rewards, ddof=1) / np.sqrt(n) if n > 1 else 0,  # 95% CI
        'pop_mean': np.mean(pops),
        'pop_std': np.std(pops, ddof=1) if n > 1 else 0,
        'n': n,
    }

print("=" * 70)
print("SUMMARY TABLE")
print("=" * 70)
print(f"{'Condition':<12} | {'Reward (mean +/- std)':<25} | {'95% CI':<20} | {'Population':<15}")
print("-" * 70)
for cond in CONDITIONS:
    if cond not in summary:
        print(f"{cond:<12} | {'(no data)':<25} |")
        continue
    s = summary[cond]
    ci_lo = s['reward_mean'] - s['reward_ci']
    ci_hi = s['reward_mean'] + s['reward_ci']
    print(f"{cond:<12} | {s['reward_mean']:>8.1f} +/- {s['reward_std']:<8.1f} | "
          f"[{ci_lo:.1f}, {ci_hi:.1f}] | "
          f"{s['pop_mean']:.1f} +/- {s['pop_std']:.1f}")

In [ ]:
# Cell 9: Statistical Tests

print("=" * 70)
print("STATISTICAL TESTS")
print("=" * 70)

def cohens_d(x, y):
    """Compute Cohen's d effect size."""
    nx, ny = len(x), len(y)
    if nx < 2 or ny < 2:
        return 0.0
    pooled_std = np.sqrt(((nx-1)*np.var(x, ddof=1) + (ny-1)*np.var(y, ddof=1)) / (nx+ny-2))
    return (np.mean(x) - np.mean(y)) / pooled_std if pooled_std > 0 else 0

adaptive_rewards = [r['total_reward'] for r in by_condition.get('adaptive', [])]
on_rewards = [r['total_reward'] for r in by_condition.get('field_on', [])]
off_rewards = [r['total_reward'] for r in by_condition.get('field_off', [])]

# ADAPTIVE vs ON
if len(adaptive_rewards) >= 2 and len(on_rewards) >= 2:
    t_ao, p_ao = stats.ttest_ind(adaptive_rewards, on_rewards, equal_var=False)
    d_ao = cohens_d(adaptive_rewards, on_rewards)
    print(f"\nADAPTIVE vs FIELD_ON:")
    print(f"  Welch's t = {t_ao:.4f}, p = {p_ao:.4f}")
    print(f"  Cohen's d = {d_ao:.4f} ({'small' if abs(d_ao)<0.5 else 'medium' if abs(d_ao)<0.8 else 'large'})")
    print(f"  Direction: {'ADAPTIVE wins' if np.mean(adaptive_rewards) > np.mean(on_rewards) else 'FIELD_ON wins'}")
else:
    p_ao, d_ao = 1.0, 0.0
    print("\nADAPTIVE vs FIELD_ON: Not enough data")

# ADAPTIVE vs OFF
if len(adaptive_rewards) >= 2 and len(off_rewards) >= 2:
    t_af, p_af = stats.ttest_ind(adaptive_rewards, off_rewards, equal_var=False)
    d_af = cohens_d(adaptive_rewards, off_rewards)
    print(f"\nADAPTIVE vs FIELD_OFF:")
    print(f"  Welch's t = {t_af:.4f}, p = {p_af:.4f}")
    print(f"  Cohen's d = {d_af:.4f} ({'small' if abs(d_af)<0.5 else 'medium' if abs(d_af)<0.8 else 'large'})")
    print(f"  Direction: {'ADAPTIVE wins' if np.mean(adaptive_rewards) > np.mean(off_rewards) else 'FIELD_OFF wins'}")
else:
    p_af, d_af = 1.0, 0.0
    print("\nADAPTIVE vs FIELD_OFF: Not enough data")

# ON vs OFF
if len(on_rewards) >= 2 and len(off_rewards) >= 2:
    t_of, p_of = stats.ttest_ind(on_rewards, off_rewards, equal_var=False)
    d_of = cohens_d(on_rewards, off_rewards)
    print(f"\nFIELD_ON vs FIELD_OFF:")
    print(f"  Welch's t = {t_of:.4f}, p = {p_of:.4f}")
    print(f"  Cohen's d = {d_of:.4f}")
else:
    print("\nFIELD_ON vs FIELD_OFF: Not enough data")

# Winner
all_means = []
for cond in CONDITIONS:
    rewards = [r['total_reward'] for r in by_condition.get(cond, [])]
    if rewards:
        all_means.append((np.mean(rewards), cond))
if all_means:
    winner = max(all_means, key=lambda x: x[0])
    print(f"\n*** WINNER: {winner[1].upper()} (mean reward = {winner[0]:.1f}) ***")

In [ ]:
# Cell 10: Bar Chart Comparison

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
bar_colors = ['#2ecc71', '#e74c3c', '#3498db']  # green, red, blue

# (a) Reward
ax = axes[0]
means = [summary.get(c, {}).get('reward_mean', 0) for c in CONDITIONS]
cis = [summary.get(c, {}).get('reward_ci', 0) for c in CONDITIONS]
bars = ax.bar(CONDITIONS, means, yerr=cis, color=bar_colors, capsize=8, edgecolor='black')
for bar, m in zip(bars, means):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(cis) + 0.5,
            f'{m:.1f}', ha='center', fontsize=10)
ax.set_ylabel('Total Reward')
ax.set_title('(a) Reward Comparison (mean +/- 95% CI)')

# (b) Population
ax = axes[1]
means = [summary.get(c, {}).get('pop_mean', 0) for c in CONDITIONS]
stds = [summary.get(c, {}).get('pop_std', 0) for c in CONDITIONS]
bars = ax.bar(CONDITIONS, means, yerr=stds, color=bar_colors, capsize=8, edgecolor='black')
ax.set_ylabel('Final Population')
ax.set_title('(b) Population Comparison (mean +/- std)')
ax.axhline(64, color='red', linestyle='--', alpha=0.5, label='Max capacity')
ax.legend()

plt.tight_layout()
plt.savefig(os.path.join(DRIVE_BASE, 'comparison.png'), dpi=150)
plt.show()

In [ ]:
# Cell 11: Gate Analysis Over Training (Adaptive Only)
# Note: Gate metrics during training are not captured by ParallelTrainer.
# This cell shows placeholder / future analysis.

adaptive_runs = by_condition.get('adaptive', [])
if not adaptive_runs:
    print("No adaptive runs completed yet.")
else:
    print(f"Adaptive runs: {len(adaptive_runs)}")
    print("")
    print("Note: ParallelTrainer doesn't currently log gate metrics during training.")
    print("Gate behavior is analyzed via final gate_bias_values in Cell 12.")
    print("")
    print("To analyze gate evolution over training, you would need to:")
    print("1. Add gate metric logging to parallel_train.py single_seed_train_step()")
    print("2. Capture 'gate/mean' and 'gate/channel_X_mean' in all_metrics dict")
    print("3. Re-run training with updated code")

In [ ]:
# Cell 12: Gate Distribution Histogram (Bimodality Check) - CRITICAL

# Initialize for scope safety (used in Cell 13)
gate_arr = None
all_gate_bias_values = []

# Collect final gate bias values from all adaptive runs
for run in by_condition.get('adaptive', []):
    if 'gate_bias_values' in run:
        all_gate_bias_values.append(run['gate_bias_values'])

if not all_gate_bias_values:
    print("No gate bias values collected. Skipping histogram.")
    print("(This is expected if adaptive runs haven't completed yet)")
else:
    gate_arr = np.concatenate(all_gate_bias_values, axis=0)  # (n_agents_total, num_channels)
    print(f"Collected gate biases from {len(all_gate_bias_values)} runs, {gate_arr.shape[0]} agents total")

    fig, axes = plt.subplots(2, 2, figsize=(12, 10))

    for ch in range(4):
        ax = axes[ch // 2, ch % 2]
        gate_ch = gate_arr[:, ch]
        ax.hist(gate_ch, bins=20, edgecolor='black', alpha=0.7)
        ax.axvline(0.0, color='red', linestyle='--', alpha=0.5, label='Bias=0 (neutral)')
        ax.set_xlabel('Gate Bias Value')
        ax.set_ylabel('Count (agents)')
        ax.set_title(f'Channel {ch} Final Gate Bias Distribution')

        # Check for bimodality: peaks at extremes
        low_count = np.sum(gate_ch < -0.5)
        high_count = np.sum(gate_ch > 0.5)
        if low_count > len(gate_ch)*0.2 and high_count > len(gate_ch)*0.2:
            ax.text(0.5, 0.95, 'BIMODAL (specialization!)',
                    transform=ax.transAxes, ha='center', fontsize=10, color='green', weight='bold')
        ax.legend()

    plt.suptitle('Final Gate Bias Distribution (Bimodal = Agents Specialize!)', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.savefig(os.path.join(DRIVE_BASE, 'gate_bias_distribution.png'), dpi=150)
    plt.show()

    # Inter-agent variance analysis
    print("=" * 60)
    print("INTER-AGENT GATE BIAS VARIANCE ANALYSIS")
    print("=" * 60)
    for ch in range(4):
        gate_ch = gate_arr[:, ch]
        var = np.var(gate_ch)
        print(f"Channel {ch}: mean={np.mean(gate_ch):.3f}, var={var:.4f} "
              f"({'high variance = differentiation!' if var > 0.05 else 'low variance = uniform'})")

In [ ]:
# Cell 13: Conclusion

print("=" * 60)
print("EXPERIMENT COMPLETE")
print("=" * 60)
print()
print("Key questions answered:")
print()

# 1. Winner determination
print("1. Does ADAPTIVE beat both ON and OFF?")
all_means = []
for cond in CONDITIONS:
    rewards = [r['total_reward'] for r in by_condition.get(cond, [])]
    if rewards:
        all_means.append((np.mean(rewards), cond))
if all_means:
    winner_name = max(all_means, key=lambda x: x[0])[1]
    print(f"   WINNER: {winner_name.upper()}")
    if 'p_ao' in dir() and 'p_af' in dir():
        print(f"   ADAPTIVE vs ON: p={p_ao:.4f}, d={d_ao:.4f}")
        print(f"   ADAPTIVE vs OFF: p={p_af:.4f}, d={d_af:.4f}")
else:
    print("   (No data available)")
print()

# 2. Gate bias direction
print("2. Do gate biases evolve toward negative (ignore) or positive (use)?")
if gate_arr is not None:
    overall_mean = np.mean(gate_arr)
    direction = ("leaning toward IGNORE field" if overall_mean < -0.2 else
                 "leaning toward USE field" if overall_mean > 0.2 else "neutral")
    print(f"   Overall mean gate bias = {overall_mean:.3f} ({direction})")
else:
    print("   (No gate data available)")
print()

# 3. Per-channel patterns
print("3. Different patterns per channel?")
print("   See per-channel histograms above.")
print()

# 4. Specialization evidence
print("4. Evidence of specialization (bimodal gate distribution)?")
if gate_arr is not None:
    for ch in range(4):
        gate_ch = gate_arr[:, ch]
        low = np.sum(gate_ch < -0.5) / len(gate_ch)
        high = np.sum(gate_ch > 0.5) / len(gate_ch)
        if low > 0.2 and high > 0.2:
            print(f"   Channel {ch}: BIMODAL ({low:.0%} low, {high:.0%} high)")
        else:
            print(f"   Channel {ch}: Unimodal")
else:
    print("   (No gate data available)")
print()

print("Next steps:")
print("- If ADAPTIVE wins: run full 30-seed experiment")
print("- If gates are bimodal: agents are specializing!")
print("- If gate biases are negative: field may be noise")
print("- If gate biases are positive: field is useful")